# Reading Movies

In [283]:
import ast
import csv
selected_trailers = []
with open('selected_trailers.csv') as f:
    reader = csv.DictReader(f)
    lst = list(reader)
    for item in lst:
        selected_trailers.append(item)

In [284]:
len(selected_trailers)

1549

In [285]:
selected_trailers[0]

{'uniqueID': '0',
 'movie_name': 'Avengers: Endgame',
 'num_trailers': '2',
 'trailer_id': '0',
 'video_url': 'https://www.youtube.com/watch?v=TcMBFSGVi1c',
 'video_title': "Marvel Studios' Avengers: Endgame - Official Trailer",
 'channel': 'Marvel Entertainment',
 'official': '1',
 'teaser': '',
 'days_diff': '-43',
 'trailer_release': '3/14/2019',
 'trailer_view': '153817402',
 'trailer_like': '3388847',
 'trailer_comment': '262164',
 'most_recent': '1',
 'oldest': '0',
 'most_liked': '1',
 'most_viewed': '1',
 'most_comment': '0'}

In [286]:
movies = []
with open('Output.csv') as f:
    reader = csv.DictReader(f)
    lst = list(reader)
    for item in lst:
        item["query_results"] = ast.literal_eval(item["query_results"])
        movies.append(item)

In [287]:
len(movies)

1967

In [288]:
workers_logs = {}

founded = False
with open("application.log.bk", "r") as f:
  for line in f.readlines():


    if "Api_key" in line:
      worker_number = line.split("Worker Number")[1].split("Started,")[0].strip()
    elif "Worker Number" in line:
      worker_number = line.split("Worker Number")[1].split(",")[0].strip()
    else:
      continue

    if worker_number not in workers_logs.keys():
      workers_logs[worker_number] = []
    workers_logs[worker_number].append(line)


processed_movies = {}
for worker_number in workers_logs:
  for log in workers_logs[worker_number]:
    if "Processing" in log:
      uniqueID = log.split("Processing: Movie UniqueID")[1].split(",")[0].strip()
      traier_id = log.split("Trailer ID")[1].split("\n")[0].strip()
      if uniqueID not in processed_movies.keys():
        processed_movies[uniqueID] = {}
    if "From total comment count" in log:
      processed_movies[uniqueID][traier_id] = log

counter = 0
for processed_movie in processed_movies:
  for result in processed_movies[processed_movie]:
    sub_total1, sub_total2 = processed_movies[processed_movie][result].split("From total comment count, must delete total_posterior_comments_count: ")[1].split(" + total_posterior_reply_count: ")
    sub_total1 = int(sub_total1)
    sub_total2 = int(sub_total2.split(" \n")[0])
    processed_movies[processed_movie][result] = sub_total1 + sub_total2

print("Amount of movies processsed {}".format(len(processed_movies)))
print("Amount of results processsed {}".format(counter))


Amount of movies processsed 1011
Amount of results processsed 0


In [289]:
for processed_movie in processed_movies:
    for trailer_id in processed_movies[processed_movie]:
        for selected_trailer in selected_trailers:
            if selected_trailer["trailer_id"] == trailer_id and selected_trailer["uniqueID"] == processed_movie:
                selected_trailer["trailer_comment"] = int(selected_trailer["trailer_comment"]) - processed_movies[processed_movie][trailer_id]
                if selected_trailer["trailer_comment"] < 0:
                    selected_trailer["trailer_comment"] = 0


In [290]:
selected_trailers[0]

{'uniqueID': '0',
 'movie_name': 'Avengers: Endgame',
 'num_trailers': '2',
 'trailer_id': '0',
 'video_url': 'https://www.youtube.com/watch?v=TcMBFSGVi1c',
 'video_title': "Marvel Studios' Avengers: Endgame - Official Trailer",
 'channel': 'Marvel Entertainment',
 'official': '1',
 'teaser': '',
 'days_diff': '-43',
 'trailer_release': '3/14/2019',
 'trailer_view': '153817402',
 'trailer_like': '3388847',
 'trailer_comment': 206098,
 'most_recent': '1',
 'oldest': '0',
 'most_liked': '1',
 'most_viewed': '1',
 'most_comment': '0'}

In [291]:
with open("./selected_trailers_comments_processed.csv", "w") as f:
    f.write(
        "uniqueID,movie_name,num_trailers,trailer_id,video_url,video_title,channel,official,teaser,days_diff,trailer_release,trailer_view,trailer_like,trailer_comment,most_recent,oldest,most_liked,most_viewed,most_comment")
    f.write("\r\n")
    for selected_trailer in selected_trailers:
        for key in selected_trailer:
            f.write(str(selected_trailer[key])+",")
        f.write("\r\n")

In [292]:
movie_id_results = {}
for trailer in selected_trailers:
    if trailer["uniqueID"] not in movie_id_results.keys():
        movie_id_results[trailer["uniqueID"]] = []
    movie_id_results[trailer["uniqueID"]].append(trailer)

In [293]:
movie_id_results["0"]

[{'uniqueID': '0',
  'movie_name': 'Avengers: Endgame',
  'num_trailers': '2',
  'trailer_id': '0',
  'video_url': 'https://www.youtube.com/watch?v=TcMBFSGVi1c',
  'video_title': "Marvel Studios' Avengers: Endgame - Official Trailer",
  'channel': 'Marvel Entertainment',
  'official': '1',
  'teaser': '',
  'days_diff': '-43',
  'trailer_release': '3/14/2019',
  'trailer_view': '153817402',
  'trailer_like': '3388847',
  'trailer_comment': 206098,
  'most_recent': '1',
  'oldest': '0',
  'most_liked': '1',
  'most_viewed': '1',
  'most_comment': '0'},
 {'uniqueID': '0',
  'movie_name': 'Avengers: Endgame',
  'num_trailers': '2',
  'trailer_id': '1',
  'video_url': 'https://www.youtube.com/watch?v=hA6hldpSTF8',
  'video_title': "Marvel Studios' Avengers - Official Trailer",
  'channel': 'Marvel Entertainment',
  'official': '1',
  'teaser': '',
  'days_diff': '-140',
  'trailer_release': '12/7/2018',
  'trailer_view': '107481220',
  'trailer_like': '3366631',
  'trailer_comment': 264627

In [294]:
new_movies = []
for movie in movies:
    if movie["uniqueID"] in movie_id_results.keys():
        results = movie_id_results[movie["uniqueID"]]
        movie["results"] = results
        new_movies.append(movie)
movies = new_movies
len(movies)

1011

In [295]:
movies[0]

{'uniqueID': '0',
 'movie_name': 'Avengers: Endgame',
 'url_bom': 'https://www.boxofficemojo.com/release/rl3059975681/',
 'success': '1',
 'gross_dom': '858373000.0',
 'gross_int': '1939128328.0',
 'gross_worldwide': '2797501328.0',
 'distributor': 'Walt Disney Studios Motion Pictures',
 'opening_box': '357115007.0',
 'opening_theater': '4662.0',
 'budget': '356000000.0',
 'release_start': '4/26/2019',
 'release_end': '9/12/2019',
 'mpaa': 'PG-13',
 'run_time': '181.0',
 'Action': '1',
 'Adventure': '1',
 'Drama': '1',
 'Sci-Fi': '1',
 'Fantasy': '0',
 'Animation': '0',
 'Family': '0',
 'Musical': '0',
 'Horror': '0',
 'Comedy': '0',
 'Thriller': '0',
 'Romance': '0',
 'Mystery': '0',
 'Crime': '0',
 'War': '0',
 'Music': '0',
 'Sport': '0',
 'Biography': '0',
 'History': '0',
 'Documentary': '0',
 'Western': '0',
 'Film-Noir': '0',
 'release_length': '250.0',
 'url_imdb': 'https://imdb.com/title/tt4154796',
 'summary': "After the devastating events of Avengers: Infinity War, the unive

In [296]:
new_movies = []
for movie in movies:
    max_comments = -1
    max_comments_id = None
    for result in movie["results"]:
        if int(result["trailer_comment"]) > max_comments:
            max_comments = int(result["trailer_comment"])
            max_comments_id = result["video_url"]

    if max_comments_id == None:
        print("Error. {}".format(movie["uniqueID"]))

    results = []
    for result in movie["results"]:
        result["most_comment"] = '0'
        if result["video_url"] == max_comments_id:
            result["most_comment"] = '1'
        results.append(result)
    movie["results"] = results
    new_movies.append(movie)
movies = new_movies

In [297]:
movies[0]

{'uniqueID': '0',
 'movie_name': 'Avengers: Endgame',
 'url_bom': 'https://www.boxofficemojo.com/release/rl3059975681/',
 'success': '1',
 'gross_dom': '858373000.0',
 'gross_int': '1939128328.0',
 'gross_worldwide': '2797501328.0',
 'distributor': 'Walt Disney Studios Motion Pictures',
 'opening_box': '357115007.0',
 'opening_theater': '4662.0',
 'budget': '356000000.0',
 'release_start': '4/26/2019',
 'release_end': '9/12/2019',
 'mpaa': 'PG-13',
 'run_time': '181.0',
 'Action': '1',
 'Adventure': '1',
 'Drama': '1',
 'Sci-Fi': '1',
 'Fantasy': '0',
 'Animation': '0',
 'Family': '0',
 'Musical': '0',
 'Horror': '0',
 'Comedy': '0',
 'Thriller': '0',
 'Romance': '0',
 'Mystery': '0',
 'Crime': '0',
 'War': '0',
 'Music': '0',
 'Sport': '0',
 'Biography': '0',
 'History': '0',
 'Documentary': '0',
 'Western': '0',
 'Film-Noir': '0',
 'release_length': '250.0',
 'url_imdb': 'https://imdb.com/title/tt4154796',
 'summary': "After the devastating events of Avengers: Infinity War, the unive

In [298]:
keys = ""
for movie in movies:
    for key in movie:
        #print(key)
        if key == "query_results":
            continue
        if key == "results":
            for result in movie["results"]:
                for result_key in result:
                    if result_key == "uniqueID" or \
                        result_key == "movie_name" or \
                        result_key == "video_title" or \
                        result_key == "channel" or \
                        result_key == "days_diff":
                        continue
                    else:
                        keys = keys + "{},".format(result_key)

                break
        else:
            keys = keys + "{},".format(key)
    break
print(keys)

uniqueID,movie_name,url_bom,success,gross_dom,gross_int,gross_worldwide,distributor,opening_box,opening_theater,budget,release_start,release_end,mpaa,run_time,Action,Adventure,Drama,Sci-Fi,Fantasy,Animation,Family,Musical,Horror,Comedy,Thriller,Romance,Mystery,Crime,War,Music,Sport,Biography,History,Documentary,Western,Film-Noir,release_length,url_imdb,summary,user_raiting,user_reviews,critic_rating,critic_reviews,writers,directors,movie_star,query_url,no_prerelease,num_trailers,trailer_id,video_url,official,teaser,trailer_release,trailer_view,trailer_like,trailer_comment,most_recent,oldest,most_liked,most_viewed,most_comment,


In [299]:
for movie in movies:
    movie["summary"] = movie["summary"].replace(",", "\,")
    break
print(movies[0])

{'uniqueID': '0', 'movie_name': 'Avengers: Endgame', 'url_bom': 'https://www.boxofficemojo.com/release/rl3059975681/', 'success': '1', 'gross_dom': '858373000.0', 'gross_int': '1939128328.0', 'gross_worldwide': '2797501328.0', 'distributor': 'Walt Disney Studios Motion Pictures', 'opening_box': '357115007.0', 'opening_theater': '4662.0', 'budget': '356000000.0', 'release_start': '4/26/2019', 'release_end': '9/12/2019', 'mpaa': 'PG-13', 'run_time': '181.0', 'Action': '1', 'Adventure': '1', 'Drama': '1', 'Sci-Fi': '1', 'Fantasy': '0', 'Animation': '0', 'Family': '0', 'Musical': '0', 'Horror': '0', 'Comedy': '0', 'Thriller': '0', 'Romance': '0', 'Mystery': '0', 'Crime': '0', 'War': '0', 'Music': '0', 'Sport': '0', 'Biography': '0', 'History': '0', 'Documentary': '0', 'Western': '0', 'Film-Noir': '0', 'release_length': '250.0', 'url_imdb': 'https://imdb.com/title/tt4154796', 'summary': "After the devastating events of Avengers: Infinity War\\, the universe is in ruins. With the help of rem

In [300]:
final_records = []
for movie in movies:
    for result in movie["results"]:
        final_record = {}
        for movie_key in movie:
            if movie_key == "query_results" or movie_key == "results":
                continue
            else:
                final_record[movie_key] = movie[movie_key]

        for result_key in result:
            if result_key == "uniqueID" or \
                result_key == "movie_name" or \
                result_key == "video_title" or \
                result_key == "channel" or \
                result_key == "days_diff":
                continue
            else:
                final_record[result_key] = result[result_key]
        final_records.append(final_record)



In [301]:
keys = final_records[0].keys()

In [302]:
movies = []
with open('Output.csv') as f:
    reader = csv.DictReader(f)
    lst = list(reader)
    for item in lst:
        item["query_results"] = ast.literal_eval(item["query_results"])
        movies.append(item)

for movie in movies:
    found = False
    for final_record in final_records:
        if movie["uniqueID"] == final_record["uniqueID"]:
            found = True
            break
    if not found:
        final_record = {}
        if movie["no_prerelease"] == "1":
            for movie_key in movie:
                if movie_key == "query_results" or movie_key == "results":
                    continue
                else:
                    final_record[movie_key] = movie[movie_key]
            final_record["num_trailers"] = 0
            final_records.append(final_record)


In [303]:
for movie in movies:
    found = False
    for final_record in final_records:
        if movie["uniqueID"] == final_record["uniqueID"]:
            found = True
            break
    if not found:
        final_record = {}
        if movie["no_prerelease"] == "0":
            for movie_key in movie:
                if movie_key == "query_results" or movie_key == "results":
                    continue
                else:
                    final_record[movie_key] = movie[movie_key]
            final_record["num_trailers"] = 0
            final_records.append(final_record)

In [304]:
with open('final_dataset.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(final_records)